<a href="https://colab.research.google.com/github/jigsawfallingintoplace/KAUST/blob/master/English_to_Spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install laserembeddings

     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 860kB 6.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=4096e919baa03bdeb24497337bc6679e5b1dd1bc9e9c56e31a87ed36539c41b1
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import csv
from laserembeddings import Laser
import torch
from torch import optim, nn

laser = Laser()

trainfile = "/content/2018-E-c-En-train.txt"
testfile = "/content/2018-E-c-Es-test-gold.txt"
devfile = "/content/2018-E-c-Es-dev.txt"
savepath = "/content/LASERSentiment.model"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def file_to_data(file):
    with open(file) as f:
        reader = csv.reader(f, delimiter="\t")
        data = list(reader)
    return data

traindata = file_to_data(trainfile)
testdata = file_to_data(testfile)
devdata = file_to_data(testfile)


def get_all_tweets(data): 
    return [d[1] for d in data[1:]]

def get_label_lists(data):
    return [[int(x) for x in d[2:]] for d in data[1:]]

def get_label_tensors(data):
    label_tensors = []
    for d in data[1:]:
        tmp = torch.zeros(11)
        for i in range(11):
            if d[2 + i] == '1':
                tmp[i] = 1
        label_tensors.append(tmp)
    return label_tensors

train_tweets = get_all_tweets(traindata)
train_embeddings = laser.embed_sentences(train_tweets, lang='en')
test_tweets = get_all_tweets(testdata)
test_embeddings = laser.embed_sentences(test_tweets, lang='es')
dev_tweets = get_all_tweets(devdata)
dev_embeddings = laser.embed_sentences(dev_tweets, lang='es')

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.layer_1 = nn.Linear(1024, 512)
        self.layer_2 = nn.Linear(512, 512)
        self.layer_3 = nn.Linear(512, 512)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(512)
        self.batchnorm3 = nn.BatchNorm1d(512)
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        # x = self.dropout(x)
        return x

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.layer_4 = nn.Linear(512, 512)
        self.layer_out = nn.Linear(512, 11) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm4 = nn.BatchNorm1d(512)
    def forward(self, x):
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        # x = nn.Softmax(dim=1)(x)
        return x

In [ ]:
class LanguageClassifier(nn.Module):
    def __init__(self):
        super(LanguageClassifier, self).__init__()
        self.layer_4 = nn.Linear(512, 512)
        self.layer_out = nn.Linear(512, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm4 = nn.BatchNorm1d(512)
    def forward(self, x):
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        # x = nn.Softmax(dim=1)(x)
        return x

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_embeddings_tensors = torch.from_numpy(train_embeddings)
label_tensors = get_label_tensors(traindata)
assert len(train_embeddings_tensors) == len(label_tensors)

train_dataset = TensorDataset(train_embeddings_tensors, torch.stack(label_tensors))
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
train_iter_source = iter(train_dataloader)


test_label_tensors = get_label_tensors(testdata)
y_true = torch.stack(test_label_tensors)

test_dataset = TensorDataset(torch.from_numpy(test_embeddings), y_true)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

train_iter_target = iter(test_dataloader)

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
lambd = 0.01
learning_rate = 0.005
Q_learning_rate = 0.005
F = FeatureExtractor()
P = SentimentClassifier()
Q = LanguageClassifier()
F, P, Q = F.to(device), P.to(device), Q.to(device)
optimizer = optim.Adam(list(F.parameters()) + list(P.parameters()), lr=learning_rate)
q_optimizer = optim.Adam(Q.parameters(), lr=Q_learning_rate)

In [ ]:
# net.load_state_dict(torch.load(savepath))

In [ ]:
def freeze(net):
    for p in net.parameters():
        p.requires_grad = False

def unfreeze(net):
    for p in net.parameters():
        p.requires_grad = True

def get_batch_source():
  global train_iter_source
  try:
    return next(train_iter_source)
  except:
    train_iter_source = iter(train_dataloader)
    return next(train_iter_source)

def get_batch_target():
  global train_iter_target
  try:
    return next(train_iter_target)
  except:
    train_iter_target = iter(test_dataloader)
    return next(train_iter_target)

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score, f1_score, jaccard_score


scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=60, verbose=True)


dev_label_tensors = get_label_tensors(devdata)
dev_y_true = torch.stack(dev_label_tensors)
up = 0

dev_dataset = TensorDataset(torch.from_numpy(dev_embeddings), dev_y_true)
dev_dataloader = DataLoader(dev_dataset, batch_size=10, shuffle=False)

loss_diagram = []
num_epochs = 100
q_iter = 5
cnt = 0
clip_lower = -0.01
clip_upper = 0.01
running_loss = 0
jac = 0
while (jac < 0.45):
    freeze(P)
    freeze(F)
    unfreeze(Q)
    for q in range(q_iter):
      for p in Q.parameters():
        p.data.clamp_(clip_lower, clip_upper)
      
      Q.zero_grad()

      X_source, _ = get_batch_source()
      X_target, _ = get_batch_target()
      feature_source = F(X_source)
      feature_target = F(X_target)
      loss_q = torch.mean(-Q(feature_source)) + torch.mean(Q(feature_target))
      q_optimizer.zero_grad()
      loss_q.backward()
      q_optimizer.step()

    unfreeze(F)
    unfreeze(P)
    freeze(Q)

    for p in Q.parameters():
        p.data.clamp_(clip_lower, clip_upper) 

    F.zero_grad()
    P.zero_grad()

    X_source, Y_source = get_batch_source()
    X_target, Y_target = get_batch_target()
    feature_source = F(X_source)
    sentiment_source = P(feature_source)
    language_source = Q(feature_source)

    feature_target = F(X_target)
    language_target = Q(feature_target)

    loss_sentiment = loss_function(sentiment_source, Y_source)
    loss_sentiment.backward(retain_graph=True)
    loss_language = lambd*(torch.mean(language_source) - torch.mean(language_target))
    loss_language.backward(retain_graph=True)

    optimizer.step()

    
        

    all_list = []
    for x, y in dev_dataloader:
      guess = P(F(x))
      all_list.append(guess)
    concat_tensor = torch.cat(all_list)
    dev_y_score = concat_tensor.detach().numpy()
    dev_y_pred = dev_y_score.copy()
    for i in range(dev_y_pred.shape[0]):
      for j in range(dev_y_pred.shape[1]):
        if (dev_y_pred[i][j] >= 0.5):
          dev_y_pred[i][j] = 1
        else:
          dev_y_pred[i][j] = 0
    jac = jaccard_score(dev_y_true, dev_y_pred, average='samples')
    print("Jaccard Score:", jac)
    scheduler.step(jac)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Jaccard Score: 0.026512497080121464
Jaccard Score: 0.03907965428638169
Jaccard Score: 0.07431090866619948
Jaccard Score: 0.07753445456669002
Jaccard Score: 0.07849801448259752
Jaccard Score: 0.12477224947442188
Jaccard Score: 0.154350619014249
Jaccard Score: 0.17526278906797474
Jaccard Score: 0.1680506890913338
Jaccard Score: 0.15074164914739546
Jaccard Score: 0.13441952814762903
Jaccard Score: 0.13859495444989486
Jaccard Score: 0.13517285680915672
Jaccard Score: 0.1383204858677879
Jaccard Score: 0.14006073347348752
Jaccard Score: 0.14644358794674142
Jaccard Score: 0.15005839757066106
Jaccard Score: 0.1634314412520439
Jaccard Score: 0.17515183368371873
Jaccard Score: 0.18985050221910765
Jaccard Score: 0.20491707544966126
Jaccard Score: 0.21127073113758468
Jaccard Score: 0.21741999532819434
Jaccard Score: 0.21152768044849335
Jaccard Score: 0.2102254146227517
Jaccard Score: 0.22013548236393365
Jaccard Score: 0.22053258584442886
Jaccard Score: 0.22585260453165146
Jaccard Score: 0.23258000

KeyboardInterrupt: ignored

In [ ]:
net = lambda x: P(F(x))

In [ ]:
en_testfile = "/content/2018-E-c-En-test-gold.txt"

en_testdata = file_to_data(en_testfile)
en_test_tweets = get_all_tweets(en_testdata)
en_test_embeddings = laser.embed_sentences(en_test_tweets, lang='en')

test_label_tensors = get_label_tensors(en_testdata)
y_true = torch.stack(test_label_tensors)

en_test_dataset = TensorDataset(torch.from_numpy(en_test_embeddings), y_true)
test_dataloader = DataLoader(en_test_dataset, batch_size=100, shuffle=False)

In [ ]:
THRESHOLD = 0.5
up = 0

all_list = []

for x, y in test_dataloader:
    guess = net(x)
    all_list.append(guess)
    # print(guess)
    # values, indices = torch.topk(guess, 2)
    # print("values:", values)
    # print("indices:", indices)
    for i in range(x.shape[0]):
        for j in range(11):
            if (guess[i][j] >= THRESHOLD and y[i][j] == 1) or (guess[i][j] < THRESHOLD and y[i][j] == 0):
              up += 1

concat_tensor = torch.cat(all_list)
y_score = concat_tensor.detach().numpy()
print(concat_tensor)


print("ACCURACY:", up / (len(test_label_tensors)*11))

tensor([[  0.7187,  -4.6658,   1.9812,  ...,  -1.1565,  -7.7492,  -9.0842],
        [  0.4551,  -2.1471,   0.6134,  ...,  -1.7455,  -4.0128,  -5.4623],
        [  3.7854,  -4.7268,   2.1217,  ...,  -0.5551,  -6.8569,  -7.2609],
        ...,
        [  8.9609, -10.3167,   5.1475,  ...,  -0.9895, -10.8832, -16.0266],
        [ -2.7824,  -1.2962,  -2.0994,  ...,   1.0403,  -2.7024,  -3.4429],
        [  1.7984,  -2.9886,   0.9447,  ...,  -2.3721,  -3.2036,  -6.7177]],
       grad_fn=<CatBackward>)
ACCURACY: 0.8424781723339563


In [ ]:
torch.save(net.state_dict(), savepath)

In [ ]:
lrap_score = label_ranking_average_precision_score(y_true, y_score)
print("LRAP Score:", lrap_score)

LRAP Score: 0.7651428614968053


In [ ]:
from sklearn.metrics import f1_score, jaccard_score
y_pred = y_score.copy()
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
    if (y_pred[i][j] >= 0.5):
      y_pred[i][j] = 1
    else:
      y_pred[i][j] = 0
f1score = f1_score(y_true, y_pred, average=None)
print("F1 Score:", f1score)
f1_micro = f1_score(y_true, y_pred, average='micro')
print("F1 Micro Score:", f1_micro)
f1_macro = f1_score(y_true, y_pred, average='macro')
print("F1 Macro Score:", f1_macro)
jac = jaccard_score(y_true, y_pred, average='samples')
print("Jaccard Score:", jac)
print(y_pred)

F1 Score: [0.66603325 0.12252964 0.62851505 0.56828194 0.72594397 0.34361233
 0.55264624 0.13995485 0.56850962 0.06521739 0.03636364]
F1 Micro Score: 0.5636349586585272
F1 Macro Score: 0.4016007204607273
Jaccard Score: 0.4356587618170926
[[1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import hamming_loss
hl = hamming_loss(y_true, y_pred)
print(hl)

0.1603113057546933


In [ ]:
torch.save(P.state_dict(), '/content/P_en_0712.model')
torch.save(F.state_dict(), '/content/F_en_0712.model')
torch.save(Q.state_dict(), '/content/Q_en_0712.model')